# Installing Dependencies for LangChain with Groq LLMs

In [2]:
# Install all necessary dependencies for LangChain with Groq integration
#!pip install langchain langchain-groq langchain-core python-dotenv
#!pip install sentence-transformers
#!pip install pinecone-client

In [3]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import Pinecone
from langchain_core.messages import HumanMessage
from langchain.chains import SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer

from pinecone import Pinecone

In [4]:
# Load environment variables
load_dotenv()

True

In [5]:
# Setting up the LLM -- LLM Wrappers
llm = ChatGroq(
    api_key=os.environ.get("GROQ_API_KEY"),
    model_name="llama3-70b-8192"  # You can also use "mixtral-8x7b-32768" or other available models
)

In [6]:
# Setting up the Prompt_Template -- Prompts
template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variable=["concept"],
    template=template,
)

In [7]:
prompt

PromptTemplate(input_variables=['concept'], input_types={}, partial_variables={}, template='\nYou are an expert data scientist with an expertise in building deep learning models.\nExplain the concept of {concept} in a couple of lines\n')

In [8]:
response = llm.invoke([HumanMessage(content=prompt.format(concept="regularization"))])
response

AIMessage(content='Regularization is a technique used in deep learning to prevent overfitting by adding a penalty term to the loss function, which discourages the model from fitting the training data too closely, and instead encourages it to generalize better to new, unseen data. This is achieved by adding a term that measures the complexity of the model, such as the magnitude of the weights, to the loss function, which helps to prevent the model from becoming too complex and overfitting to the training data.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 37, 'total_tokens': 135, 'completion_time': 0.354125012, 'prompt_time': 0.000888584, 'queue_time': 0.052829845, 'total_time': 0.355013596}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--637a6385-fe19-47c3-9f76-f785ffc26bfd-0', usage_metadata={'input_tokens': 37, 'output_token

In [9]:
response = llm.invoke([HumanMessage(content=prompt.format(concept="autoencoder"))])
response

AIMessage(content='An autoencoder is a type of neural network that learns to compress and reconstruct its input data by mapping it to a lower-dimensional representation, called the bottleneck or latent representation, and then reconstructing the original input from this representation. In essence, an autoencoder is trained to copy its input to its output, thereby learning to identify the most important features of the data and ignore the noise or irrelevant information.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 38, 'total_tokens': 119, 'completion_time': 0.293842422, 'prompt_time': 0.000899734, 'queue_time': 0.054964585, 'total_time': 0.294742156}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c07a8f3c-573a-4934-a1d7-5231e0812d56-0', usage_metadata={'input_tokens': 38, 'output_tokens': 81, 'total_tokens': 119})

In [10]:
"""
Setting up Chains -- 
A chain takes a language model and a prompt template and combines them into a interface 
that thakes an 'input' from the user and 'outputs' an answer from an LLM
"""

"\nSetting up Chains -- \nA chain takes a language model and a prompt template and combines them into a interface \nthat thakes an 'input' from the user and 'outputs' an answer from an LLM\n"

In [11]:
chain = LLMChain(llm = llm, prompt = prompt)

# Run the cahin only specifying the input variable
print(chain.run("autoencoder"))

C:\Users\mbkhn\AppData\Local\Temp\ipykernel_21580\4138082450.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt)
C:\Users\mbkhn\AppData\Local\Temp\ipykernel_21580\4138082450.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run("autoencoder"))


An autoencoder is a type of neural network that learns to compress and reconstruct its input data by mapping it to a lower-dimensional latent representation, called the bottleneck or encoding, and then reconstructing the original input from this representation. This process allows the autoencoder to learn a compact and efficient representation of the data, which can be useful for tasks such as dimensionality reduction, anomaly detection, and generative modeling.


In [12]:
second_prompt = PromptTemplate(
    input_variables = ["ml_concept"],
    template = "Turn the concept description of{ml_concepts} and explain it to me like I'm five",
)

chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [41]:
overall_chain = SimpleSequentialChain(chains = [chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain
explanation= overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of neural network that learns to compress and reconstruct its input data by mapping it to a lower-dimensional representation, called the bottleneck or latent representation, and then reconstructing the original data from this representation. This process allows the autoencoder to learn a compact and meaningful representation of the data, which can be used for tasks such as dimensionality reduction, anomaly detection, and generative modeling.
Oh boy, are you ready for a fun explanation?

Imagine you have a big box full of toys, and inside that box, you have lots of different toys like blocks, dolls, cars, and puzzles. Now, imagine you want to send this box of toys to your friend who lives far, far away.

But, there's a problem! The box is too big and heavy to send, and it would cost a lot of money to mail it. So, you need to find a way to make the box smaller and lighter, without losing any of the toys inside.

Tha

In [43]:
"""
# Embeddings and VectorStores
"""

'\n# Embeddings and VectorStores\n'

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
)

texts = text_splitter.create_documents([explanation])

In [47]:
texts[0].page_content

'Oh boy, are you ready for a fun explanation?'

In [49]:
# embedding model -- from HuggingFace
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding = HuggingFaceEmbeddings(model_name=model_name)

In [50]:
#query_embedding = embedding(texts[0].page_content)

In [51]:
#query_embedding

In [52]:
#dimension = query_embedding.shape[0]

In [57]:
from pinecone import Pinecone, ServerlessSpec

# ✅ Initialize Pinecone client
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)  # Replace with your actual Pinecone API key

# Delete existing index
pc.delete_index("langchain-quickstart")

# ✅ Create index
pc.create_index(
    name="langchain-quickstart",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [58]:
from langchain_community.vectorstores import Pinecone as LangPinecone

index_name = "langchain-quickstart"
search = LangPinecone.from_documents(
    documents=texts,
    embedding=embedding,
    index_name=index_name
)

In [67]:
query = "What is an autoencoder?"
results = search.similarity_search(query, k=4)
for r in results:
    print(r.page_content)

And that's autoencoders in a nutshell!
That's kind of like what an autoencoder does. An autoencoder is a special kind of computer program
autoencoder has a special set of instructions that says, "Hey, I know what all these toys look
But here's the magic part! The autoencoder can then take this tiny box and use it to rebuild the


In [68]:
results

[Document(metadata={}, page_content="And that's autoencoders in a nutshell!"),
 Document(metadata={}, page_content="That's kind of like what an autoencoder does. An autoencoder is a special kind of computer program"),
 Document(metadata={}, page_content='autoencoder has a special set of instructions that says, "Hey, I know what all these toys look'),
 Document(metadata={}, page_content="But here's the magic part! The autoencoder can then take this tiny box and use it to rebuild the")]

In [69]:
print(texts[0])

page_content='Oh boy, are you ready for a fun explanation?' metadata={'text': 'Oh boy, are you ready for a fun explanation?'}


In [73]:
"""
# Agents
"""

'\n# Agents\n'

In [113]:
#pip install langchain-experimental
#!pip uninstall langchain -y
#!pip install langchain --upgrade --no-cache-dir

Found existing installation: langchain 0.3.26
Uninstalling langchain-0.3.26:
  Successfully uninstalled langchain-0.3.26
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 6.8 MB/s eta 0:00:00


In [120]:
from langchain_groq import ChatGroq
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.agents.agent_toolkits import create_python_agent

In [111]:
import langchain
print(langchain.__version__)

0.3.25


In [130]:
# 1. Your LLM
llm = ChatGroq(model_name="llama3-8b-8192", temperature=0)

# 2. Build the agent
agent = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [131]:
# 3. Run!
print(agent.run("What is 3 + 3?"))



> Entering new AgentExecutor chain...
Thought: I need to calculate the sum of 3 and 3.
Action: Python_REPL
Action Input: print(3 + 3)
Observation: 6

Thought:Thought: I now know the final answer
Final Answer: 6

> Finished chain.
6


In [134]:
agent.run("Find the roots (zeros) if the quadratic function 3 * x * * 2 + 2 * x - 1")



> Entering new AgentExecutor chain...
Thought: To find the roots of the quadratic function 3*x^2 + 2*x - 1, I can use the quadratic formula. The quadratic formula is given by (-b ± √(b^2 - 4ac)) / 2a, where a, b, and c are the coefficients of the quadratic function.

Action: Python_REPL
Action Input: from sympy import symbols, Eq, solve
x = symbols('x')
eq = Eq(3*x**2 + 2*x - 1, 0)
roots = solve(eq, x)
print(roots)

Observation: [-1, 1/3]

Thought:Question: Find the roots (zeros) if the quadratic function 3 * x * * 2 + 2 * x - 1

Thought: To find the roots of the quadratic function 3*x^2 + 2*x - 1, I can use the quadratic formula. The quadratic formula is given by (-b ± √(b^2 - 4ac)) / 2a, where a, b, and c are the coefficients of the quadratic function.

Action: Python_REPL
Action Input: from sympy import symbols, Eq, solve
x = symbols('x')
eq = Eq(3*x**2 + 2*x - 1, 0)
roots = solve(eq, x)
print(roots)

Observation: [-1, 1/3]

Thought:Question: Find the roots (zeros) if the quadrati

'The roots of the quadratic function 3*x^2 + 2*x - 1 are -1 and 1/3.'